https://ysyblog.tistory.com/119

In [1]:
import pandas as pd
import numpy as np

# p-value and vif check
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

C:\Users\dltmf\anaconda3\envs\pycaret\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
dataset = pd.read_csv('Z:/공모전/날씨빅데이터콘테스트(공유)/data/test/hospital_gender_longform_0805_test2.csv', encoding='cp949')
female  = dataset[dataset.sex == 2]
male    = dataset[dataset.sex == 1]

display(female.head())
display(male.head())

,area,tma,yyyy,mm,dd,weekday,sum_gsr,sum_rn,avg_ps,max_ps,...,diff_hum,min_temp,max_temp,avg_temp,diff_temp,pm10_3b,pm10_7b,pm10_31b,sex,frequency
1,강원,2012-01-01,2012,1,1,6,0.0,0.173611,1023.886667,1026.533333,...,65.0,-7.300000,5.0,-2.344048,12.300000,87.662500,30.158333,15.678261,2,3.0
3,강원,2012-01-02,2012,1,2,0,0.0,0.050000,1026.620000,1029.166667,...,68.0,-12.600000,4.0,-3.409048,16.600000,70.629167,34.325889,16.787500,2,0.0
5,강원,2012-01-03,2012,1,3,1,0.0,0.027778,1021.546667,1025.766667,...,72.0,-13.000000,5.0,-4.009383,18.000000,89.595833,54.223703,9.787500,2,0.0
7,강원,2012-01-04,2012,1,4,2,0.0,0.000000,1024.880000,1028.520000,...,70.0,-15.200000,3.0,-6.007792,18.200000,86.279167,73.445833,34.941667,2,2.0
9,강원,2012-01-05,2012,1,5,3,0.0,0.007042,1029.871429,1031.500000,...,78.0,-17.200001,4.0,-6.291558,21.200001,70.744444,87.662500,44.916667,2,1.0


,area,tma,yyyy,mm,dd,weekday,sum_gsr,sum_rn,avg_ps,max_ps,...,diff_hum,min_temp,max_temp,avg_temp,diff_temp,pm10_3b,pm10_7b,pm10_31b,sex,frequency
0,강원,2012-01-01,2012,1,1,6,0.0,0.173611,1023.886667,1026.533333,...,65.0,-7.300000,5.0,-2.344048,12.300000,87.662500,30.158333,15.678261,1,3.0
2,강원,2012-01-02,2012,1,2,0,0.0,0.050000,1026.620000,1029.166667,...,68.0,-12.600000,4.0,-3.409048,16.600000,70.629167,34.325889,16.787500,1,2.0
4,강원,2012-01-03,2012,1,3,1,0.0,0.027778,1021.546667,1025.766667,...,72.0,-13.000000,5.0,-4.009383,18.000000,89.595833,54.223703,9.787500,1,0.0
6,강원,2012-01-04,2012,1,4,2,0.0,0.000000,1024.880000,1028.520000,...,70.0,-15.200000,3.0,-6.007792,18.200000,86.279167,73.445833,34.941667,1,1.0
8,강원,2012-01-05,2012,1,5,3,0.0,0.007042,1029.871429,1031.500000,...,78.0,-17.200001,4.0,-6.291558,21.200001,70.744444,87.662500,44.916667,1,2.0


In [30]:
dataset.yyyy.unique()

array([2012, 2013, 2014, 2015], dtype=int64)

In [17]:
X = dataset.drop(['tma','D/R','frequency'], axis=1)
y = dataset['frequency']

# bias를 위한 상수항 추가
X_data = sm.add_constant(X, has_constant = "add")
X_data

,const,area,yyyy,mm,dd,weekday,sum_gsr,sum_rn,avg_ps,max_ps,...,avg_hum,diff_hum,min_temp,max_temp,avg_temp,diff_temp,pm10_3b,pm10_7b,pm10_31b,sex
0,1.0,강원,2012,1,1,6,0.0,0.173611,1023.886667,1026.533333,...,58.333333,65.0,-7.3,5.0,-2.344048,12.3,87.662500,30.158333,15.678261,1
1,1.0,강원,2012,1,1,6,0.0,0.173611,1023.886667,1026.533333,...,58.333333,65.0,-7.3,5.0,-2.344048,12.3,87.662500,30.158333,15.678261,2
2,1.0,강원,2012,1,2,0,0.0,0.050000,1026.620000,1029.166667,...,60.490476,68.0,-12.6,4.0,-3.409048,16.6,70.629167,34.325889,16.787500,1
3,1.0,강원,2012,1,2,0,0.0,0.050000,1026.620000,1029.166667,...,60.490476,68.0,-12.6,4.0,-3.409048,16.6,70.629167,34.325889,16.787500,2
4,1.0,강원,2012,1,3,1,0.0,0.027778,1021.546667,1025.766667,...,59.992131,72.0,-13.0,5.0,-4.009383,18.0,89.595833,54.223703,9.787500,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49669,1.0,충북,2015,12,29,1,0.0,0.000000,656.200000,751.775000,...,62.439189,60.0,-14.0,4.0,-2.886486,18.0,84.616667,53.163258,44.287500,2
49670,1.0,충북,2015,12,30,2,0.0,0.919231,1030.200000,1033.800000,...,70.440315,54.0,-10.0,5.3,-0.944369,15.3,43.112500,48.222283,54.106250,1
49671,1.0,충북,2015,12,30,2,0.0,0.919231,1030.200000,1033.800000,...,70.440315,54.0,-10.0,5.3,-0.944369,15.3,43.112500,48.222283,54.106250,2
49672,1.0,충북,2015,12,31,3,0.0,0.096154,1030.850000,1034.400000,...,70.658784,55.0,-4.0,6.1,0.642230,10.1,33.789583,74.573958,56.070265,1


In [20]:
X_data

,const,area,yyyy,mm,dd,weekday,sum_gsr,sum_rn,avg_ps,max_ps,...,avg_hum,diff_hum,min_temp,max_temp,avg_temp,diff_temp,pm10_3b,pm10_7b,pm10_31b,sex
0,1.0,강원,2012,1,1,6,0.0,0.173611,1023.886667,1026.533333,...,58.333333,65.0,-7.3,5.0,-2.344048,12.3,87.662500,30.158333,15.678261,1
1,1.0,강원,2012,1,1,6,0.0,0.173611,1023.886667,1026.533333,...,58.333333,65.0,-7.3,5.0,-2.344048,12.3,87.662500,30.158333,15.678261,2
2,1.0,강원,2012,1,2,0,0.0,0.050000,1026.620000,1029.166667,...,60.490476,68.0,-12.6,4.0,-3.409048,16.6,70.629167,34.325889,16.787500,1
3,1.0,강원,2012,1,2,0,0.0,0.050000,1026.620000,1029.166667,...,60.490476,68.0,-12.6,4.0,-3.409048,16.6,70.629167,34.325889,16.787500,2
4,1.0,강원,2012,1,3,1,0.0,0.027778,1021.546667,1025.766667,...,59.992131,72.0,-13.0,5.0,-4.009383,18.0,89.595833,54.223703,9.787500,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49669,1.0,충북,2015,12,29,1,0.0,0.000000,656.200000,751.775000,...,62.439189,60.0,-14.0,4.0,-2.886486,18.0,84.616667,53.163258,44.287500,2
49670,1.0,충북,2015,12,30,2,0.0,0.919231,1030.200000,1033.800000,...,70.440315,54.0,-10.0,5.3,-0.944369,15.3,43.112500,48.222283,54.106250,1
49671,1.0,충북,2015,12,30,2,0.0,0.919231,1030.200000,1033.800000,...,70.440315,54.0,-10.0,5.3,-0.944369,15.3,43.112500,48.222283,54.106250,2
49672,1.0,충북,2015,12,31,3,0.0,0.096154,1030.850000,1034.400000,...,70.658784,55.0,-4.0,6.1,0.642230,10.1,33.789583,74.573958,56.070265,1


In [31]:
# area labelencoding
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
dataset['area'] = encoder.fit_transform(dataset.area)
dataset.head()

,area,tma,yyyy,mm,dd,weekday,sum_gsr,sum_rn,avg_ps,max_ps,...,diff_hum,min_temp,max_temp,avg_temp,diff_temp,pm10_3b,pm10_7b,pm10_31b,sex,frequency
0,0,2012-01-01,2012,1,1,6,0.0,0.173611,1023.886667,1026.533333,...,65.0,-7.3,5.0,-2.344048,12.3,87.662500,30.158333,15.678261,1,3.0
1,0,2012-01-01,2012,1,1,6,0.0,0.173611,1023.886667,1026.533333,...,65.0,-7.3,5.0,-2.344048,12.3,87.662500,30.158333,15.678261,2,3.0
2,0,2012-01-02,2012,1,2,0,0.0,0.050000,1026.620000,1029.166667,...,68.0,-12.6,4.0,-3.409048,16.6,70.629167,34.325889,16.787500,1,2.0
3,0,2012-01-02,2012,1,2,0,0.0,0.050000,1026.620000,1029.166667,...,68.0,-12.6,4.0,-3.409048,16.6,70.629167,34.325889,16.787500,2,0.0
4,0,2012-01-03,2012,1,3,1,0.0,0.027778,1021.546667,1025.766667,...,72.0,-13.0,5.0,-4.009383,18.0,89.595833,54.223703,9.787500,1,0.0


In [32]:
encoder.classes_

array(['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '세종', '울산',
       '인천', '전남', '전북', '제주', '충남', '충북'], dtype=object)

In [33]:
dataset.columns

Index(['area', 'tma', 'yyyy', 'mm', 'dd', 'weekday', 'sum_gsr', 'sum_rn',
       'avg_ps', 'max_ps', 'min_ps', 'so2', 'o3', 'pm10', 'elderly_pop',
       'total_pop', 'elderly_ratio', 'avg_age', 'min_max_ps', 'D/R',
       'heat_wave', 'cold_wave', 'min_hum', 'max_hum', 'avg_hum', 'diff_hum',
       'min_temp', 'max_temp', 'avg_temp', 'diff_temp', 'pm10_3b', 'pm10_7b',
       'pm10_31b', 'sex', 'frequency'],
      dtype='object')

In [34]:
full_model = sm.OLS.from_formula("frequency ~ " + "+".join(dataset.drop(['tma','D/R','frequency'], axis=1).columns), data=dataset)
full_result = full_model.fit()

full_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              frequency   R-squared:                       0.443
Model:                            OLS   Adj. R-squared:                  0.443
Method:                 Least Squares   F-statistic:                     1318.
Date:                Sat, 06 Aug 2022   Prob (F-statistic):               0.00
Time:                        02:08:53   Log-Likelihood:                -81480.
No. Observations:               49674   AIC:                         1.630e+05
Df Residuals:                   49643   BIC:                         1.633e+05
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        42.5736     11.629      3.661      0.000      19.781      65.366
area             -0.0211      0.001    -15.910      0.000      -0.024      -0.019
yyyy             -0.0221      0.006     -3.793      0.000      -0.034      -0.011
mm               -0.0025      0.002     -1.280      0.200      -0.006       0.001
dd               -0.0078      0.001    -12.145      0.000      -0.009      -0.007
weekday          -0.0989      0.003    -35.100      0.000      -0.104      -0.093
sum_gsr          -0.0016      0.002     -1.019      0.308      -0.005       0.001
sum_rn           -0.0011      0.001     -1.998      0.046      -0.002   -2.05e-05
avg_ps            0.0001      0.000      1.021      0.307      -0.000       0.000
max_ps        -1.227e+04   2.58e+04     -0.476      0.634   -6.28e+04    3.83e+04
min_ps         1.227e+04   2.58e+04      0.476      0.634   -3.83e+04    6.28e+04
so2              18.3144      3.084      5.939      0.000      12.270      24.359
o3               -0.2611      0.527     -0.495      0.620      -1.295       0.772
pm10              0.0001      0.000      0.344      0.731      -0.000       0.001
elderly_pop    7.247e-07   1.89e-07      3.841      0.000    3.55e-07    1.09e-06
total_pop      6.093e-07   2.01e-08     30.333      0.000     5.7e-07    6.49e-07
elderly_ratio     0.0352      0.006      6.238      0.000       0.024       0.046
avg_age           0.0612      0.010      6.140      0.000       0.042       0.081
min_max_ps     1.227e+04   2.58e+04      0.476      0.634   -3.83e+04    6.28e+04
heat_wave         0.0363      0.025      1.474      0.140      -0.012       0.085
cold_wave        -0.0289      0.032     -0.904      0.366      -0.092       0.034
min_hum           0.2531      0.532      0.476      0.634      -0.789       1.295
max_hum          -0.2520      0.532     -0.474      0.636      -1.294       0.790
avg_hum           0.0008      0.002      0.540      0.589      -0.002       0.004
diff_hum          0.2538      0.532      0.477      0.633      -0.789       1.296
min_temp          0.0557      0.141      0.397      0.692      -0.220       0.331
max_temp         -0.0772      0.141     -0.549      0.583      -0.353       0.198
avg_temp          0.0228      0.009      2.456      0.014       0.005       0.041
diff_temp         0.0675      0.140      0.481      0.631      -0.208       0.343
pm10_3b           0.0006      0.000      2.378      0.017       0.000       0.001
pm10_7b           0.0002      0.000      0.669      0.503      -0.000       0.001
pm10_31b          0.0009      0.000      3.613      0.000       0.000       0.001
sex              -0.3868      0.016    -24.744      0.000      -0.417      -0.356
==============================================================================
Omnibus:                     8411.150   Durbin-Watso

In [42]:
# 범주형변수 처리
feature_nm = dataset.drop(['tma','D/R','frequency', 'pm10','pm10_3b', 'pm10_31b',
                           'area','yyyy','mm','dd','weekday','heat_wave','cold_wave','elderly_pop'], axis=1).columns
feature_nms = [name for name in feature_nm] + ["C(area)", "C(yyyy)","C(mm)","C(dd)","C(weekday)","C(heat_wave)","C(cold_wave)"]
# feature_nms

full_model2 = sm.OLS.from_formula("frequency ~ 0 + " + "+".join(feature_nms), data=dataset)
full_result2 = full_model2.fit()

full_result2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              frequency   R-squared:                       0.466
Model:                            OLS   Adj. R-squared:                  0.465
Method:                 Least Squares   F-statistic:                     485.8
Date:                Sat, 06 Aug 2022   Prob (F-statistic):               0.00
Time:                        02:22:09   Log-Likelihood:                -80462.
No. Observations:               49674   AIC:                         1.611e+05
Df Residuals:                   49584   BIC:                         1.619e+05
Df Model:                          89                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
C(area)[0]            2.7292      0.626      4.357      0.000       1.501       3.957
C(area)[1]            8.5609      1.249      6.852      0.000       6.112      11.010
C(area)[2]            4.1725      0.660      6.320      0.000       2.878       5.467
C(area)[3]            3.7918      0.655      5.793      0.000       2.509       5.075
C(area)[4]            2.5320      0.590      4.294      0.000       1.376       3.688
C(area)[5]            3.1155      0.637      4.891      0.000       1.867       4.364
C(area)[6]            2.5192      0.601      4.193      0.000       1.342       3.697
C(area)[7]            3.8405      0.687      5.586      0.000       2.493       5.188
C(area)[8]            7.0414      1.091      6.456      0.000       4.904       9.179
C(area)[9]            1.4938      0.586      2.548      0.011       0.344       2.643
C(area)[10]           2.2570      0.603      3.741      0.000       1.075       3.439
C(area)[11]           3.1461      0.641      4.909      0.000       1.890       4.402
C(area)[12]           3.1368      0.630      4.976      0.000       1.901       4.372
C(area)[13]           3.0088      0.622      4.835      0.000       1.789       4.228
C(area)[14]           1.7962      0.593      3.030      0.002       0.634       2.958
C(area)[15]           2.8630      0.620      4.616      0.000       1.647       4.079
C(area)[16]           2.7169      0.614      4.425      0.000       1.513       3.920
C(yyyy)[T.2013]      -0.0021      0.017     -0.120      0.904      -0.036       0.031
C(yyyy)[T.2014]       0.0006      0.020      0.032      0.975      -0.039       0.040
C(yyyy)[T.2015]       0.0380      0.023      1.647      0.100      -0.007       0.083
C(mm)[T.2]            0.0019      0.028      0.066      0.947      -0.053       0.057
C(mm)[T.3]           -0.0151      0.031     -0.478      0.632      -0.077       0.047
C(mm)[T.4]           -0.0213      0.038     -0.560      0.575      -0.096       0.053
C(mm)[T.5]           -0.0891      0.047     -1.910      0.056      -0.180       0.002
C(mm)[T.6]           -0.1311      0.052     -2.512      0.012      -0.233      -0.029
C(mm)[T.7]           -0.1196      0.057     -2.091      0.037      -0.232      -0.007
C(mm)[T.8]           -0.1356      0.058     -2.331      0.020      -0.250      -0.022
C(mm)[T.9]           -0.1301      0.051     -2.571      0.010      -0.229      -0.031
C(mm)[T.10]          -0.0877      0.042     -2.090      0.037      -0.170      -0.005
C(mm)[T.11]          -0.0403      0.033     -1.223      0.221      -0.105       0.024
C(mm)[T.12]          -0.0335      0.028     -1.196      0.232      -0.088       0.021
C(dd)[T.2]           -0.9040      0.043    -21.083      0.000      -0.988      -0.820
C(dd)[T.3]           -0.9737      0.043    -22.674      0.000      -1.058

In [46]:
feature_nm2 = dataset.drop(['tma','D/R','frequency', 'pm10','pm10_3b', 'pm10_31b','elderly_pop'], axis=1).columns

# DataFrame 내 숫자 정수표현 설정
pd.options.display.float_format = '{:.4f}'.format
# pd.reset_option('display.float_format') # 재설정(reset)

# VIF를 통한 다중공선성 확인

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(dataset[feature_nm2].values, i) for i in range(dataset[feature_nm2].shape[1])]
vif["features"] = dataset[feature_nm2].columns
vif.sort_values('VIF Factor')

C:\Users\dltmf\anaconda3\envs\pycaret\lib\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,VIF Factor,features
17,1.1323,cold_wave
16,1.1543,heat_wave
5,1.1587,sum_gsr
6,1.3649,sum_rn
12,2.5775,total_pop
4,3.2771,weekday
3,4.2238,dd
0,4.9298,area
26,5.2017,pm10_7b
2,6.2456,mm


## female

In [6]:
# area labelencoding
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
# dataset['area'] = encoder.fit_transform(dataset.area)
female['area'] = encoder.fit_transform(female.area)
male['area'] = encoder.transform(male.area)

female

C:\Users\dltmf\AppData\Local\Temp\ipykernel_40432\2141544312.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  female['area'] = encoder.fit_transform(female.area)
C:\Users\dltmf\AppData\Local\Temp\ipykernel_40432\2141544312.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  male['area'] = encoder.transform(male.area)


,area,tma,yyyy,mm,dd,weekday,sum_gsr,sum_rn,avg_ps,max_ps,...,diff_hum,min_temp,max_temp,avg_temp,diff_temp,pm10_3b,pm10_7b,pm10_31b,sex,frequency
1,0,2012-01-01,2012,1,1,6,0.0,0.173611,1023.886667,1026.533333,...,65.0,-7.300000,5.0,-2.344048,12.300000,87.662500,30.158333,15.678261,2,3.0
3,0,2012-01-02,2012,1,2,0,0.0,0.050000,1026.620000,1029.166667,...,68.0,-12.600000,4.0,-3.409048,16.600000,70.629167,34.325889,16.787500,2,0.0
5,0,2012-01-03,2012,1,3,1,0.0,0.027778,1021.546667,1025.766667,...,72.0,-13.000000,5.0,-4.009383,18.000000,89.595833,54.223703,9.787500,2,0.0
7,0,2012-01-04,2012,1,4,2,0.0,0.000000,1024.880000,1028.520000,...,70.0,-15.200000,3.0,-6.007792,18.200000,86.279167,73.445833,34.941667,2,2.0
9,0,2012-01-05,2012,1,5,3,0.0,0.007042,1029.871429,1031.500000,...,78.0,-17.200001,4.0,-6.291558,21.200001,70.744444,87.662500,44.916667,2,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49665,16,2015-12-27,2015,12,27,6,0.0,0.000000,653.066667,655.466667,...,57.0,-8.000000,1.0,-3.581757,9.000000,74.573958,73.132292,26.595833,2,0.0
49667,16,2015-12-28,2015,12,28,0,0.0,0.000000,655.500000,656.700000,...,55.0,-12.000000,2.0,-3.746396,14.000000,45.588542,63.497554,42.664991,2,2.0
49669,16,2015-12-29,2015,12,29,1,0.0,0.000000,656.200000,751.775000,...,60.0,-14.000000,4.0,-2.886486,18.000000,84.616667,53.163258,44.287500,2,2.0
49671,16,2015-12-30,2015,12,30,2,0.0,0.919231,1030.200000,1033.800000,...,54.0,-10.000000,5.3,-0.944369,15.300000,43.112500,48.222283,54.106250,2,1.0


In [7]:
X_female = female.drop(['tma','D/R','frequency', 'sex'], axis=1)
y_female = female['frequency']

# bias를 위한 상수항 추가
X_f_data = sm.add_constant(X_female, has_constant = "add")
X_f_data

,const,area,yyyy,mm,dd,weekday,sum_gsr,sum_rn,avg_ps,max_ps,...,max_hum,avg_hum,diff_hum,min_temp,max_temp,avg_temp,diff_temp,pm10_3b,pm10_7b,pm10_31b
1,1.0,0,2012,1,1,6,0.0,0.173611,1023.886667,1026.533333,...,95.0,58.333333,65.0,-7.300000,5.0,-2.344048,12.300000,87.662500,30.158333,15.678261
3,1.0,0,2012,1,2,0,0.0,0.050000,1026.620000,1029.166667,...,98.0,60.490476,68.0,-12.600000,4.0,-3.409048,16.600000,70.629167,34.325889,16.787500
5,1.0,0,2012,1,3,1,0.0,0.027778,1021.546667,1025.766667,...,100.0,59.992131,72.0,-13.000000,5.0,-4.009383,18.000000,89.595833,54.223703,9.787500
7,1.0,0,2012,1,4,2,0.0,0.000000,1024.880000,1028.520000,...,85.0,52.270563,70.0,-15.200000,3.0,-6.007792,18.200000,86.279167,73.445833,34.941667
9,1.0,0,2012,1,5,3,0.0,0.007042,1029.871429,1031.500000,...,93.0,51.928571,78.0,-17.200001,4.0,-6.291558,21.200001,70.744444,87.662500,44.916667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49665,1.0,16,2015,12,27,6,0.0,0.000000,653.066667,655.466667,...,80.0,48.593468,57.0,-8.000000,1.0,-3.581757,9.000000,74.573958,73.132292,26.595833
49667,1.0,16,2015,12,28,0,0.0,0.000000,655.500000,656.700000,...,80.0,52.663288,55.0,-12.000000,2.0,-3.746396,14.000000,45.588542,63.497554,42.664991
49669,1.0,16,2015,12,29,1,0.0,0.000000,656.200000,751.775000,...,90.0,62.439189,60.0,-14.000000,4.0,-2.886486,18.000000,84.616667,53.163258,44.287500
49671,1.0,16,2015,12,30,2,0.0,0.919231,1030.200000,1033.800000,...,94.0,70.440315,54.0,-10.000000,5.3,-0.944369,15.300000,43.112500,48.222283,54.106250


In [ ]:
full_model = sm.OLS.from_formula("frequency ~ " + "+".join(dataset.drop(['tma','D/R','frequency'], axis=1).columns), data=dataset)
full_result = full_model.fit()

full_result.summary()